In [ ]:
import itk
print(itk.__version__)

In [ ]:
import vtk
print(vtk.__version__)

In [3]:
# Paths
Data = '../Data/'
scan1_name = 'case6_gre1'
scan2_name = 'case6_gre2'

def getFrom(path):
    scan = itk.imread(path)
    caster = itk.CastImageFilter[scan, itk.Image[itk.F, 3]].New()
    caster.SetInput(scan)
    caster.Update()
    scan = caster.GetOutput()
    
    img = itk.GetArrayFromImage(scan)
    min_val = img.min()
    max_val = img.max()
    
    print(max_val)
    
    img = 255 * (img - min_val) / (max_val - min_val)
    
    scan = itk.GetImageFromArray(img)
    
    return scan

scan1 = getFrom(Data + scan1_name + ".nrrd")
scan2 = getFrom(Data + scan2_name + ".nrrd")

1734.0
1374.0


In [4]:
vtk_image1 = itk.vtk_image_from_image(scan1)
vtk_image2 = itk.vtk_image_from_image(scan2)

In [5]:
def create_false_tumor(base_image, pos: list[int], size: int):
    # Create a new image
    false_tumor = vtk.vtkImageData()
    false_tumor.DeepCopy(base_image)

    min_bound = pos - size // 2
    max_bound = pos + size // 2
    
    # Get the image data
    data = false_tumor.GetPointData().GetScalars()
    
    # Create a false tumor
    for i in range(data.GetNumberOfTuples()):
        if (min_bound < i % 256 < max_bound and min_bound < i // 256 % 256 < max_bound and min_bound < i // 256 // 256 % 256 < max_bound):
            data.SetTuple1(i, 255)
        else:
            data.SetTuple1(i, 0)
    
    return false_tumor

In [6]:
tumor_1 = create_false_tumor(vtk_image1, 128, 20)
tumor_2 = create_false_tumor(vtk_image2, 128, 40)

In [7]:
# Create merged image
def merge_segmentation(segmentation1, segmentation2, scan1):
    # Create a new image
    merged_image = vtk.vtkImageData()
    merged_image.DeepCopy(scan1)
    
    # Get the image data
    seg_data1 = segmentation1.GetPointData().GetScalars()
    seg_data2 = segmentation2.GetPointData().GetScalars()
    data_merged = merged_image.GetPointData().GetScalars()
    
    # Merge the images
    for i in range(seg_data1.GetNumberOfTuples()):
        value1 = seg_data1.GetTuple1(i)
        value2 = seg_data2.GetTuple1(i)
        if (value1 == 255):
            data_merged.SetTuple1(i, 255)
        elif (value2 == 255):
            data_merged.SetTuple1(i, 200)
        else:
            if (data_merged.GetTuple1(i) <= 20):
                data_merged.SetTuple1(i, 0)
            elif (20 < data_merged.GetTuple1(i) < 60):
                data_merged.SetTuple1(i, 100)
            else:
                data_merged.SetTuple1(i, 150)
    
    return merged_image

In [8]:
merge_image = merge_segmentation(tumor_1, tumor_2, vtk_image1)

In [9]:
# Test vtk display
def display_evolution(merged_segmentation):
    # Create a rendering window and renderer
    ren = vtk.vtkRenderer()
    renWin = vtk.vtkRenderWindow()
    renWin.AddRenderer(ren)

    # Create a renderwindowinteractor
    iren = vtk.vtkRenderWindowInteractor()
    iren.SetRenderWindow(renWin)

    # Create a volume
    volumeMapper = vtk.vtkSmartVolumeMapper()
    volumeMapper.SetInputData(merged_segmentation)
    volume = vtk.vtkVolume()
    volume.SetMapper(volumeMapper)

    # Create transfer mapping scalar value to opacity
    opacityTransferFunction = vtk.vtkPiecewiseFunction()
    opacityTransferFunction.AddPoint(0, 0.0)
    opacityTransferFunction.AddPoint(99, 0.0)
    opacityTransferFunction.AddPoint(100, 0.01)
    opacityTransferFunction.AddPoint(150, 0.01)
    opacityTransferFunction.AddPoint(199, 0.01)
    opacityTransferFunction.AddPoint(200, 0.1)
    opacityTransferFunction.AddPoint(254, 0.1)
    opacityTransferFunction.AddPoint(255, 0.5)

    # Create transfer mapping scalar value to color
    colorTransferFunction = vtk.vtkColorTransferFunction()
    colorTransferFunction.AddRGBPoint(0.0, 0.0, 0.0, 0.0)
    colorTransferFunction.AddRGBPoint(99.0, 0.0, 0.0, 0.0)
    colorTransferFunction.AddRGBPoint(100.0, 1., 0.5, 0.5)
    colorTransferFunction.AddRGBPoint(150.0, 0.95, 0.85, 0.75)
    colorTransferFunction.AddRGBPoint(192.0, 1.0, 0.0, 0.0)
    colorTransferFunction.AddRGBPoint(255.0, 0.0, 0.0, 1.0)

    # The property describes how the data will look
    volumeProperty = vtk.vtkVolumeProperty()
    volumeProperty.SetColor(colorTransferFunction)
    volumeProperty.SetScalarOpacity(opacityTransferFunction)
    volumeProperty.SetInterpolationTypeToLinear()
    # volumeProperty.ShadeOn()

    # The mapper / ray cast function know how to render the data
    volumeMapper.SetBlendModeToComposite()
    volume.SetProperty(volumeProperty)
    ren.AddViewProp(volume)

    # Set background color
    ren.SetBackground(0.7, 0.7, 0.8)

    # Set window size
    renWin.SetSize(600, 600)

    # Initialize the interactor and start the rendering loop
    iren.Initialize()
    renWin.Render()
    iren.Start()

    # Clean up
    iren.GetRenderWindow().Finalize()
    iren.TerminateApp()

In [10]:
display_evolution(merge_image)